In [ ]:
!pip install loguru
!pip install transformers --upgrade

In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict, concatenate_datasets

# 1. Charger les fichiers CSV
'''df_train = pd.read_csv('/content/french_paradetox_2500.csv', sep='\t')
df_test = pd.read_csv('/content/french_paradetox_eval_500.csv', sep='\t')

df_train = df_train.dropna()
df_test = df_test.dropna()

# 2. Concaténer les deux DataFrames
#df_combined = pd.concat([df_train, df_test], ignore_index=True)
'''
df = pd.read_csv('/content/filtered_df.csv', sep='\t')
df = df[["toxic_text","neutral_text"]]
df = df.rename(columns={
    "toxic_text": "fr_toxic_comment",
    "neutral_text": "fr_neutral_comment"
})
# 3. Convertir en HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# 4. Diviser en train/test (80% train, 20% test)
split_dataset = dataset.train_test_split(test_size=0.15, seed=42)

# 5. Créer un DatasetDict
final_dataset = DatasetDict({
    'train': split_dataset['train'],
    'test': split_dataset['test']
})

# 6. Afficher les tailles
print(f"Taille du train : {len(final_dataset['train'])}")
print(f"Taille du test  : {len(final_dataset['test'])}")

Taille du train : 4511
Taille du test  : 797


In [2]:
import datasets
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

In [3]:
hf_dataset_parallel = final_dataset
#tokenizer = AutoTokenizer.from_pretrained('Dalfaxy/mt0_xl_french_detox_v2')
tokenizer = AutoTokenizer.from_pretrained('bigscience/mt0-xl')
def preprocess_function_parallel(examples):

    toxic = ["Détoxifier" + tox for tox in examples['fr_toxic_comment']]

    inputs = tokenizer(toxic, truncation=True, max_length=128)
    labels = tokenizer(examples['fr_neutral_comment'], truncation=True, max_length=128)
    return {
        **inputs,
        'labels': labels.input_ids
    }

print(hf_dataset_parallel)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['fr_toxic_comment', 'fr_neutral_comment'],
        num_rows: 4511
    })
    test: Dataset({
        features: ['fr_toxic_comment', 'fr_neutral_comment'],
        num_rows: 797
    })
})


In [4]:
from peft import PeftModel

In [ ]:
tokenized_dataset_parallel = hf_dataset_parallel.map(preprocess_function_parallel,  batched=True,)
train_dataset = tokenized_dataset_parallel['train']
test_dataset =  tokenized_dataset_parallel['test']

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="bigscience/mt0-xl")

model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/mt0-xl")
#model = AutoModelForSeq2SeqLM.from_pretrained("Dalfaxy/mt0_xl_french_detox_v2")

ADD_LORA = True
#ADD_LORA = False

if ADD_LORA:
    peft_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        inference_mode=False,
        r=32,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=['k', 'lm_head', 'o', 'q', 'v', 'wi_0', 'wi_1', 'wo']
    )

    model = get_peft_model(model, peft_config)
    model.load_adapter("Dalfaxy/mt0_xl_french_detox_v2", adapter_name="default")
    model.print_trainable_parameters()

DEVICE = 'cuda'
model.to(DEVICE)

Map:   0%|          | 0/4511 [00:00<?, ? examples/s]

Map:   0%|          | 0/797 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/797 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/7.03G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/7.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 78,848,000 || all params: 3,821,467,648 || trainable%: 2.0633


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 2048)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 2048)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2048, out_features=32, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=32, out_features=2048, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()


In [ ]:
model.print_trainable_parameters()

trainable params: 78,848,000 || all params: 3,821,467,648 || trainable%: 2.0633


In [ ]:
from transformers import EarlyStoppingCallback

In [ ]:
print((train_dataset[0]["labels"]))

[10975, 334, 340, 9534, 5465, 15584, 263, 907, 1520, 346, 259, 369, 259, 13004, 1]


In [ ]:
BATCH_SIZE = 16 # mt0-xl
#BATCH_SIZE = 2 # mt0-xxl
OUTPUT_DIR = 'output'

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=1e-5,
    lr_scheduler_type='cosine',
    # warmup_steps=4,
    gradient_accumulation_steps=2, # mt0-xxl
    num_train_epochs=8,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.0,
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    # evaluation_strategy='steps',
    # save_strategy='steps',
    # eval_steps=2500,
    # save_steps=2500,
    bf16=True,
    report_to= [],
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()
trainer.save_model(OUTPUT_DIR + '/model')

/tmp/ipython-input-1135786684.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,1.149200,1.012801
2,0.926600,0.879356
3,0.997800,0.835416
4,0.922100,0.817387
5,0.839000,0.807976
6,0.883800,0.803322
7,0.956400,0.801809
8,0.838000,0.801674


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:222: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:222: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:222: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:222: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers fou

In [ ]:
print(f"Best config: LR={best_config[0]}, WD={best_config[1]}, BS={best_config[2]}")

Best config: LR=1e-05, WD=0.0, BS=16


In [ ]:
# Login to Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# prompt: push the model to my huggingface

# Push to Hub
# Replace 'your_username/your_model_name' with your desired repository name on Hugging Face
repo_name = "Dalfaxy/mt0_xl_french_detox_"



# Push the trained model and tokenizer to the Hub
# Use push_to_hub method of the trainer
trainer.push_to_hub(repo_name)

In [ ]:
model.push_to_hub("Dalfaxy/mt0_xl_french_detox_v3")
tokenizer.push_to_hub("Dalfaxy/mt0_xl_french_detox_v3")

README.md: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:222: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...pbzzawvxq/adapter_model.safetensors:   0%|          | 5.12MB / 2.36GB            

...

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpyqyvsaq7/spiece.model         : 100%|##########| 4.31MB / 4.31MB            

  /tmp/tmpyqyvsaq7/tokenizer.json       :   3%|2         |  433kB / 16.3MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/Dalfaxy/mt0_xl_french_detox_v3/commit/468f7d8b39c9065d157f4e214b6bb50043cd6107', commit_message='Upload tokenizer', commit_description='', oid='468f7d8b39c9065d157f4e214b6bb50043cd6107', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Dalfaxy/mt0_xl_french_detox_v3', endpoint='https://huggingface.co', repo_type='model', repo_id='Dalfaxy/mt0_xl_french_detox_v3'), pr_revision=None, pr_num=None)